#  Apple Leaf Disease Classification with CNN using Tensorflow 2.0

### Introduction.

In this notebook, we'll demonstrate how to build a convolutional neural network model to classify Apple leaf diseases. We'll use the tensorflow keras API.

### Import necessary libraries 

In [ ]:
import os
import cv2
import PIL
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from tensorflow.keras.models import Sequential, load_model, model_from_json


: 

This dataset was taken from this Kaggle link. https://www.kaggle.com/datasets/lavaman151/plantifydr-dataset

Here, we want to view a sample of our images. We'll use tf.keras image preprocessing library to load the image.

In [ ]:
img1 =image.load_img('/kaggle/input/plantifydr-dataset/PlantDiseasesDataset/Apple/train/Apple Black rot/AppleBlackRot(1000).JPG')

: 

: 

In [ ]:
plt.imshow(img1)

: 

In [ ]:
cv2.imread('/kaggle/input/plantifydr-dataset/PlantDiseasesDataset/Apple/train/Apple Black rot/AppleBlackRot(7).JPG').shape

: 

### Referencing our data directories.

In [ ]:
train_dir ='/kaggle/input/plantifydr-dataset/PlantDiseasesDataset/Apple/train'
test_dir = '/kaggle/input/plantifydr-dataset/PlantDiseasesDataset/Apple/test'
IMG_SIZE = (256, 256)
BATCH_SIZE = 16

: 

### Preprocess training image data.

Here, we'll load in our data using ImageDataGenerator Class in tensorflow. This allows us to rescale our images and split the data into train and validation data. It also loads our images in a specified batch size.

ImageDataGenerator infers the class labels from the directories name. We want to check the class labels.

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
    )
val_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
    )  

: 

In [ ]:
train_set = train_gen.flow_from_directory(
    train_dir,
    subset = 'training',
    class_mode = 'categorical',
    target_size = IMG_SIZE,
    batch_size = 16
)

: 

In [ ]:
val_set = val_gen.flow_from_directory(
   train_dir,
   subset = 'validation',
   class_mode = 'categorical',
   target_size = IMG_SIZE,
   batch_size = 16
   )

: 

In [ ]:
train_set.classes

: 

In [ ]:
train_set.class_indices

: 

### Building the model.

In [ ]:

model = tf.keras.models.Sequential([
        
        layers.InputLayer(input_shape=(256, 256, 3)),
    
        layers.Conv2D( 32, 3,padding='valid', activation='relu'),
        
        layers.MaxPooling2D(pool_size=(2,2)),
        #  #########
        
        layers.Conv2D( 64, 3,padding='valid', activation='relu'),
        
        layers.MaxPooling2D(pool_size=(2,2)),
        #  #########
        
        layers.Conv2D( 64, 3,padding='valid', activation='relu'),
        
        layers.MaxPooling2D(pool_size=(2,2)),
        #  #########
        
        layers.Conv2D( 64, 3,padding='valid', activation='relu'),
        
        layers.MaxPooling2D(pool_size=(2,2)),
        #  #########
        
        layers.Conv2D( 128, 3,padding='valid', activation='relu'),
        
        layers.MaxPooling2D(pool_size=(2,2)),
        # ##########
        
        layers.Conv2D( 64, 3,padding='valid', activation='relu'),
        
        layers.MaxPooling2D(pool_size=(2,2)),
        # ##########
        
        layers.Flatten(),
        
        layers.Dense(64, activation='relu'),
        
        layers.Dense(4, activation='softmax')
        ])

print(model.summary())



: 

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
    )

: 

We are now fitting the images into the model.

In [ ]:
final_model = model.fit(
     train_set,
     epochs=15,
     validation_data=val_set, 
     steps_per_epoch = len(train_set),
     validation_steps = len(val_set)
     )

: 

### Preprocess test image data.
In the case of the test images, we'll only rescale the images and generate the batches.

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)

: 

In [ ]:
test_set = test_gen.flow_from_directory(
    test_dir,
    class_mode = 'categorical',
    target_size = IMG_SIZE,
    batch_size = 16
)

: 

In [ ]:
final_model.params

: 

In [ ]:
final_model.history.keys()

: 

### Evaluating our model.
Here, we'll plot the accuracy and loss for both training and validation data.

In [ ]:
acc = final_model.history['accuracy']
val_acc = final_model.history['val_accuracy']

loss = final_model.history['loss']
val_loss = final_model.history['val_loss']

: 

In [ ]:
# plot the accuracy and loss
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(15), acc, label='Training Accuracy')
plt.plot(range(15), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()
# plt.savefig('AccVal_acc')

plt.subplot(1, 2, 2)
plt.plot(range(15), loss, label='Training Loss')
plt.plot(range(15), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
# plt.savefig('LossVal_loss')


: 

We want to save our model for later use so that we don't have to train it all over again to save time.

In [ ]:
model.save('./apple_leaf_cnn_model.h5')

: 

### Making predictions with test images.

In [ ]:
def predict(model, images):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i])
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = test_set.classes[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)

    return predicted_class, confidence

: 

We want to evaluate our model's performance with the test images.

In [ ]:
scores = model.evaluate(test_set, batch_size=64, verbose=2)

: 